# DPO orca-math-korean

**Dataset:**
- https://huggingface.co/datasets/microsoft/orca-math-word-problems-200k
- https://huggingface.co/datasets/kuotient/orca-math-korean-dpo-pairs

**Model:**
- https://huggingface.co/soka0000/vclm-korean-7b

In [1]:
%pip install datasets transformers hf_transfer accelerate peft trl wandb scikit-learn -Uq

Note: you may need to restart the kernel to use updated packages.


## 데이터 준비

In [2]:
from datasets import load_dataset  # HuggingFace 데이터셋 로드 함수

# DPO 용 데이터 (chosen/rejected) 학습 데이터 로드
dataset = load_dataset('kuotient/orca-math-korean-dpo-pairs', split='train')
SAMPLE_SIZE = 10000
dataset = dataset.select(range(SAMPLE_SIZE))  # 앞에서부터 10000개만 선택
print(len(dataset))
print(dataset[100])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/162M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/192848 [00:00<?, ? examples/s]

10000
{'system': '당신은 인공지능 어시스턴트입니다.', 'question': '한 배럴에는 12리터(L)와 400밀리리터(ml)의 석유, B 배럴에는 7600밀리리터(ml)의 석유가 들어 있습니다. A 배럴과 B 배럴의 석유 양을 같게 하려면 A 배럴에서 B 배럴로 몇 리터(L)를 옮겨야 합니까?', 'chosen': '먼저 모든 측정값을 동일한 단위로 변환하여 계산을 쉽게 해봅시다. 모든 것을 밀리리터(ml)로 변환하겠습니다.\n\n배럴에는 12리터와 400밀리리터가 들어 있습니다. 1리터는 1000밀리리터와 같으므로 12리터를 밀리리터로 변환할 수 있습니다:\n12리터 = 12 * 1000밀리리터 = 12000밀리리터\n\n이제 이미 밀리리터 단위로 표시된 400밀리리터를 더합니다:\n12000밀리리터 + 400밀리리터 = 12400밀리리터\n\n따라서 배럴에는 총 12400밀리리터의 석유 가 들어 있습니다.\n\nB 배럴에는 7600밀리리터의 석유가 들어 있습니다.\n\n두 배럴의 석유 양을 동일하게 하려면 두 양의 평균을 구해야 합니다:\n두 배럴의 총량 = 12400밀리리터(A배럴) + 7600밀리리터(B배럴)\n두 배럴의 총량 = 20000밀리리터\n\n이제 이 총량을 2로 나누어 각 배럴에 해당하는 양을 구합니다:\n각 배럴의 동일한 양 = 20000밀리리터 / 2\n각 배럴의 동일한 양 = 10000밀리리터\n\n현재 A 배럴에는 12400밀리리터가 있으므로, 두 배럴의 양이 각각 10000밀리리터가 되도록 일부를 B 배럴로 이동해야 합니다.\n\nA에서 B로 이동할 양 = 12400밀리리터(A 배럴) - 10000밀리리터(동일한 양)\nA에서 B로 이동할 양 = 2400밀리리터\n\n따라서 두 배럴의 석유 양을 동일하게 하기 위해 A 배럴에서 B 배럴로 2400밀리리터(또는 2.4리터)를 이동해야 합니다.', 'rejected': ' 먼저 B 배럴의 석유 양을 리터로 변환하여 A 배럴의 석유 양과 비교해야 합니다.\n\nB 배럴의

In [3]:
print(dataset[9999])

{'system': '당신은 인공지능 어시스턴트입니다.', 'question': '제임스는 일주일 동안 500개의 깡통을 모아 깡통 집을 짓기로 결심합니다. 첫날에는 50개의 깡통을 모았습니다. 둘째 날에는 첫날에 모은 깡통의 특정 배수를 모았습니다. 셋째 날에는 둘째 날에 모은 깡통보다 50개를 더 모았습니다. 그는 일주일의 나머지 날에도 매일 50개씩 같은 수의 깡통을 모았습니다. 둘째 날에 모은 깡통 수와 첫째 날에 모은 깡통 수의 비율은 얼마입니까?', 'chosen': '두 번째 날에 제임스가 수집한 깡통의 수를 첫 번째 날에 수집한 깡통의 수에 \\( x \\) 배로 표시해 봅시다. 따라서 둘째 날에 그는 \\( 50x \\) 깡통을 수집합니다.\n\n셋째 날에는 \\( 50x - 50 \\) 깡통을 수집합니다.\n\n남은 4일(4일부터 7일까지) 동안 그는 매일 50개의 깡통을 수집하여 총 4 \\times 50 = 200 개의 깡통을 수집합니다.\n\n한 주 동안 수집한 깡통의 총 개수는 매일 수집한 깡통의 합으로, 500개의 깡통과 같아야 합니다. 따라서 우리는 다음과 같습니다:\n\n\\[ 50 + 50x + (50x - 50) + 200 = 500 \\]\n\n유사한 항을 결합하면 다음과 같습니다:\n\n\\[ 50x + 50x + 200 = 500 \\]\n\n\\[ 100x + 200 = 500 \\]\n\n양쪽에서 200을 빼면 다음과 같습니다:\n\n\\[ 100x = 300 \\]\n\n양변을 100으로 나누면 다음과 같습니다:\n\n\\[ x = 3 \\]\n\n따라서 둘째 날에 수집한 깡통의 수와 첫째 날에 수집한 깡통의 수의 비율은 \\( x:1, 즉 3:1입니다.', 'rejected': ' 둘째 날에 제임스가 모은 깡통의 수를 \\( x \\)`로 표시해 봅시다.\n\n첫째 날에는 50개의 깡통을 모았습니다.\n둘째 날에는 \\( x \\)개의 깡통을 모았습니다.\n셋째 날에는 \\( x + 50 \\)개의 깡통을 모았습니

## 모델 준비

https://huggingface.co/soka0000/vclm-korean-7b

In [4]:
# Policy(학습 대상) 모델 / 토크나이저 로드
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer  # 생성형 모델 / 토크나이저 자동 로드

model_name = 'soka0000/vclm-korean-7b'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)  # 토크나이저 로드 (커스텀 코드 허용)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype = torch.bfloat16,    # 메모리 절약 및 연산 효율성
    device_map = "auto",       # CPU/GPU 자동 배치
    trust_remote_code = True   # 모델 저장소에 있는 커스텀 모델 코드 허용
)

config.json: 0.00B [00:00, ?B/s]

modeling_soka.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/soka0000/vclm-korean-7b:
- modeling_soka.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

## 데이터 전처리

In [5]:
# 원본 샘플(system/question/chosen/rejected) -> prompt/chosen/rejected 형태의 모델용 학습 데이터 형태로 바꾸는 함수
def preprocess_dpo_data(example):
    messages = [
        {"role": "system", "content": example['system']},  # 시스템 프롬프트
        {"role": "user", "content": example['question']}   # 사용자 질문
    ]

    # 모델이 기대하는 포맷으로 변환 (|im_start|system...|im_start|user...). 답변 생성 시작 지점까지 모두 포함
    prompt_style = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # DPOTrainer가 사용하기 좋은 형태
    return {
        'prompt' : prompt_style,           # 템플릿이 적용된 프롬프트
        'chosen' : example['chosen'],      # 선호 응답
        'rejected' : example['rejected'],  # 비선호 응답
    }

dataset_preprocessed = dataset.map(preprocess_dpo_data)  # 전체 데이터셋에 전처리 함수 적용

print(f'before: {dataset.column_names}')
print(f'after: {dataset_preprocessed.column_names}')  # prompt 컬럼은 추가 생성

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

before: ['system', 'question', 'chosen', 'rejected']
after: ['system', 'question', 'chosen', 'rejected', 'prompt']


In [6]:
dataset_preprocessed['prompt'][0]

'<|im_start|>system\n당신은 인공지능 어시스턴트입니다.<|im_end|>\n<|im_start|>user\n정국이 5위입니다. 정국보다 결승선을 먼저 통과한 사람의 수를 찾아보세요.<|im_end|>\n<|im_start|>assistant\n'

In [7]:
# 데이터셋 분할
# - train/val_test 비율 8:1:1

train_size = int(len(dataset_preprocessed) * 0.8)
val_size = int(len(dataset_preprocessed) * 0.1)
test_size = int(len(dataset_preprocessed) * 0.1)

train_dataset = dataset_preprocessed.select(range(train_size))  # 앞 80% train
val_dataset = dataset_preprocessed.select(range(train_size, train_size + val_size))  # 그 다음 10% val
test_dataset = dataset_preprocessed.select(range(train_size + val_size, len(dataset_preprocessed))) # 나머지는 test

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

8000
1000
1000


## BaseModel 학습전 테스트

In [8]:
# prompt(질문)를 넣고, 모델이 답변을 생성하도록 하는 함수
def generate_response(model, tokenizer, question):
    prompt = question
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)  # prompt를 토큰화 후, 장치로 이동

    with torch.no_grad():              # 추론 모드
        outputs = model.generate(      # 텍스트 생성
            **inputs,                  # input_ids / attention_mask
            max_length = 1024,         # 생성을 포함한 최대 토큰 길이
            do_sample = True,          # 샘플링 생성(매번 결과가 다르다)
            top_k = 50,                # 확률 상위 k개 토큰 중에서 선택
            top_p = 0.95,              # 누적 확률 p 범위 내 토큰 중에서 선택
            temperature = 0.5,         # 창의성
            num_return_sequences = 1,  # 답변 1개
            eos_token_id = tokenizer.eos_token_id,  # 종료 토큰
            pad_token_id = tokenizer.pad_token_id   # 패딩 토큰
        )
        generated_text = tokenizer.decode(outputs[0])      # 생성된 토큰을 문자열로 디코딩
        return generated_text.replace(prompt, '').strip()  # prompt 제거 후 답변만 반환

In [9]:
test_subset = test_dataset.select(range(3))

for i, example in enumerate(test_subset):  # 샘플 3개를 순회
    question = example['prompt']
    answer = example['chosen']    # 선호 답변(데이터셋에서는 정답)

    print(f'[질문]:\n{question}')
    print(f'\n[정답]:\n{answer}')

    generated_answer = generate_response(model, tokenizer, question)
    print(f'\n[모델 생성 답변]:\n{generated_answer}')
    print('-' * 100)

[질문]:
<|im_start|>system
당신은 인공지능 어시스턴트입니다.<|im_end|>
<|im_start|>user
알리사와 아비게일은 과학 프로젝트를 위해 빈 캔 100개를 모아야 합니다. 오늘 현재 알리사는 빈 캔을 몇 개 모았고, 아비게일은 빈 캔 43개를 모았습니다. 그들은 빈 캔을 27개 더 모아야 합니다. 지금까지 알리사가 모은 빈 캔은 몇 개인가요?<|im_end|>
<|im_start|>assistant


[정답]:
알리사가 얼마나 많은 빈 캔을 모았는지 알아내려면, 아비게일이 모은 캔의 수와 프로젝트에 필요한 총 캔 수에서 아직 모아야 할 캔의 수를 빼야 합니다.

알리사의 캔 + 아비게일의 캔 + 아직 필요한 캔 = 필요한 총 캔 수입니다.
알리사의 캔 = 필요한 총 캔 수 - (아비게일의 캔 + 아직 필요한 캔)

우리는 이것을 알고 있습니다:
필요한 총 캔 수 = 100
아비게일의 캔 = 43
아직 필요한 캔 = 27

이제 숫자를 연결할 수 있습니다:

알리사의 캔 = 100 - (43 + 27)
알리사의 캔 = 100 - 70
알리사의 캔 = 30

알리사는 지금까지 빈 캔 30개를 모았습니다.

[모델 생성 답변]:
알리사가 지금까지 모은 빈 캔의 수를 알아내기 위해, 우리는 아비게일이 모은 빈 캔의 수에 27개를 더한 다음, 알리사와 아비게일이 모으려는 총 빈 캔의 수에서 빼야 합니다: 100 - (43 + 27) = 100 - 70 = 30 그러므로, 알리사가 지금까지 모은 빈 캔은 30개입니다.<|im_end|>
----------------------------------------------------------------------------------------------------
[질문]:
<|im_start|>system
당신은 인공지능 어시스턴트입니다.<|im_end|>
<|im_start|>user
알리사와 아비게일은 과학 프로젝트를 위해 빈 캔 100개를 모아야 합니다. 오늘 현재 알리사는 빈 캔 

## DPO 학습 준비

In [10]:
# LoRA(PEFT) 적용으로 정책모델 학습 파라미터 축소 확인
from peft import LoraConfig, get_peft_model  # LoRA 설정 / PEFT 적용 함수

lora_config = LoraConfig(
    r = 8,                   # LoRA 랭크 (추가로 학습할 저차원 행렬 크기)
    lora_alpha = 16,         # LoRA 스케일 (업데이트 강도 조절)
    target_modules = ["q_proj", "v_proj"],  # LoRA에 붙일 모듈 (어텐션의 Q/V 프로젝션)
    lora_dropout = 0.05,     # 과적합 완화
    bias = 'none',           # bias 학습 제외
    task_type = 'CAUSAL_LM'  # 작업 유형 (디코더 LM)
)

# 정책 모델 (학습대상)
model = get_peft_model(model, lora_config)  # 기존 모델에 LoRA 적용 (PEFT 모델)

model.print_trainable_parameters()  # 학습가능한 파라미터 수/비율

trainable params: 2,523,136 || all params: 7,618,139,648 || trainable%: 0.0331


In [11]:
# 참조모델 : 정책모델이 얼마나 바뀌었는지 비교 기준이 되는 고정 모델
# - chosen/rejecdted 답변에 대한 생성확률 비교값 제공

# 참조모델 (학습하지 않은 기본모델)
reference_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype = torch.bfloat16,    # 메모리 절약 및 연산 효율성
    device_map = "auto",       # CPU/GPU 자동 배치
    trust_remote_code = True   # 모델 저장소에 있는 커스텀 모델 코드 허용
)

# 학습방지 처리
reference_model.eval()  # 평가모드 전환
for param in reference_model.parameters():  # 모든 파라미터 순회
    param.requires_grad = False             # gradient 계산 / 업데이트 방지

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

In [12]:
# DPOConfig 설정
from trl import DPOTrainer, DPOConfig  # DPO 학습 클래스 / 설정 클래스

hub_model_id = 'capybaraOh/vclm-korean-7b-orca-math-korean-dpo'  # 학습 완료 모델을 올릴 Hub 저장소

# DPO 학습 하이퍼파라미터/로깅/저장 설정
training_args = DPOConfig(
    output_dir='vclm-korean-7b-orca-math-korean-dpo',  # 체크포인트/로그 저장 폴더
    num_train_epochs=1,             # 전체 데이터를 1번 반복 학습
    per_device_train_batch_size=2,  # GPU 1개당 배치 크기
    gradient_accumulation_steps=4,  # 4번 누적 후 업데이트(실제 배치 효과: 2*4=8)
    learning_rate=5e-5,             # 학습률
    eval_strategy="steps",          # 일정 step마다 평가 수행
    save_strategy="steps",          # 일정 step마다 저장 수행
    logging_steps=50,               # 50 step마다 학습 로그 출력
    fp16=False,                     # fp16 비활성화(여기서는 bf16 사용)
    bf16=True,                      # bfloat16 사용(지원 GPU에서 안정적/빠름)
    tf32=True,                      # Ampere 이상에서 matmul 가속 옵션(정밀도 약간 완화)
    beta=0.1,                       # DPO의 beta(선호 강도 조절)
    max_length=512,                 # prompt+답변을 포함한 최대 길이
    max_prompt_length=256,          # prompt에 할당할 최대 길이
    remove_unused_columns=False,    # DPO에 필요한 컬럼이 제거되지 않도록 유지
    push_to_hub=True,               # 학습 결과를 Hugging Face Hub로 업로드
    hub_model_id=hub_model_id,      # 업로드할 저장소 이름
    hub_strategy="end",             # 학습 끝난 뒤 한 번만 업로드
    report_to=['wandb']             # wandb로 학습 로그 전송
)

/tmp/ipykernel_418/381938084.py:7: FutureWarning: `max_prompt_length` is deprecated and will be removed in version 0.29.0. We recommend filtering out overlong prompts from your dataset before passing it to the trainer instead of using this parameter.
  training_args = DPOConfig(


In [13]:
# DPO Trainer 생성 (정책모델 vs 참조모델 비교 학습)
dpo_trainer = DPOTrainer(
    model = model,                  # 정책모델
    ref_model = reference_model,    # 참조모델
    args = training_args,           # 학습 설정
    train_dataset = train_dataset,  # 학습 데이터 (prompt/chosen/rejected)
    eval_dataset = val_dataset,     # 검증 데이터
    processing_class = tokenizer    # 토큰화 처리
)

dpo_trainer.train()  # DPO학습 실행

Extracting prompt in train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

  ········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: capybara-ohgiraffers (capybara-ohgiraffers-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
50,0.579535,0.392952,0.702219,-0.206515,0.893000,0.908734,-179.547287,-234.314026,1.139387,1.017785
100,0.249898,0.154205,-0.024517,-3.058099,0.963000,3.033582,-186.814636,-262.829865,1.191236,1.057575
150,0.110530,0.100609,-0.676667,-6.047880,0.965000,5.371213,-193.336121,-292.727692,1.208654,1.066885
200,0.068252,0.095909,-1.238648,-7.874978,0.965000,6.636330,-198.955948,-310.998657,1.213631,1.065149
250,0.091005,0.093133,-1.170401,-8.480999,0.966000,7.310599,-198.273453,-317.058868,1.196639,1.047680
300,0.057171,0.093310,-1.267962,-9.321945,0.968000,8.053984,-199.249100,-325.468323,1.182520,1.031703
350,0.045625,0.110507,-1.798110,-11.051714,0.968000,9.253606,-204.550568,-342.765991,1.175144,1.019505
400,0.046910,0.112846,-2.375344,-12.288463,0.971000,9.913117,-210.322891,-355.133545,1.167809,1.005387
450,0.063200,0.092997,-2.104456,-11.990234,0.969000,9.885779,-207.614029,-352.151184,1.162987,1.000989
500,0.091108,0.077229,-1.583007,-11.376247,0.973000,9.793241,-202.399536,-346.011322,1.166556,1.005637


TrainOutput(global_step=1000, training_loss=0.10134319376945496, metrics={'train_runtime': 3939.7714, 'train_samples_per_second': 2.031, 'train_steps_per_second': 0.254, 'total_flos': 0.0, 'train_loss': 0.10134319376945496, 'epoch': 1.0})

In [30]:
# 학습이 중간에 끊어졌거나 혹은 자동으로 업로드되지 않은 경우
dpo_trainer.push_to_hub('Commit!')  # 현재 학습된 체크포인트/모델을 Hub에 커밋 메시지와 함께 업로드

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/capybaraOh/vclm-korean-7b-orca-math-korean-dpo/commit/b2eed7ab89b73fc43812cfec4e2058173439afcd', commit_message='Commit!', commit_description='', oid='b2eed7ab89b73fc43812cfec4e2058173439afcd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/capybaraOh/vclm-korean-7b-orca-math-korean-dpo', endpoint='https://huggingface.co', repo_type='model', repo_id='capybaraOh/vclm-korean-7b-orca-math-korean-dpo'), pr_revision=None, pr_num=None)

## DPO Model 테스트

In [15]:
test_subset = test_dataset.select(range(3))

for i, example in enumerate(test_subset):  # 샘플 3개를 순회
    question = example['prompt']
    answer = example['chosen']    # 선호 답변(데이터셋에서는 정답)

    print(f'[질문]:\n{question}')
    print(f'\n[정답]:\n{answer}')

    generated_answer = generate_response(model, tokenizer, question)
    print(f'\n[모델 생성 답변]:\n{generated_answer}')
    print('-' * 100)

[질문]:
<|im_start|>system
당신은 인공지능 어시스턴트입니다.<|im_end|>
<|im_start|>user
알리사와 아비게일은 과학 프로젝트를 위해 빈 캔 100개를 모아야 합니다. 오늘 현재 알리사는 빈 캔을 몇 개 모았고, 아비게일은 빈 캔 43개를 모았습니다. 그들은 빈 캔을 27개 더 모아야 합니다. 지금까지 알리사가 모은 빈 캔은 몇 개인가요?<|im_end|>
<|im_start|>assistant


[정답]:
알리사가 얼마나 많은 빈 캔을 모았는지 알아내려면, 아비게일이 모은 캔의 수와 프로젝트에 필요한 총 캔 수에서 아직 모아야 할 캔의 수를 빼야 합니다.

알리사의 캔 + 아비게일의 캔 + 아직 필요한 캔 = 필요한 총 캔 수입니다.
알리사의 캔 = 필요한 총 캔 수 - (아비게일의 캔 + 아직 필요한 캔)

우리는 이것을 알고 있습니다:
필요한 총 캔 수 = 100
아비게일의 캔 = 43
아직 필요한 캔 = 27

이제 숫자를 연결할 수 있습니다:

알리사의 캔 = 100 - (43 + 27)
알리사의 캔 = 100 - 70
알리사의 캔 = 30

알리사는 지금까지 빈 캔 30개를 모았습니다.

[모델 생성 답변]:
이 문제를 해결하기 위해 두 가지 정보를 알아야 합니다: 알리사가 지금까지 모은 빈캔의 수와 아비게일이 모은 총 빈캔 수입니다.

주어진 정보에 따르면:
- 아비게일은 43개의 빈캔을 모았습니다.
- 알리사와 아비게일은 함께 27개의 빈캔을 더 모아야 합니다.
- 총 빈캔 수는 100개입니다.

먼저 아비게일이 더 이상 모으지 않는 가정 하에 알리사와 아비게일이 이미 모은 총 빈캔 수를 계산합니다:

빈캔 총 개수 = 알리사가 모은 빈캔 + 아비게일이 모은 빈캔
100 - 27(더해야 할 빈캔) = 알리사가 모은 빈캔 + 43

이제 아비게일이 이미 모은 것과 더해야 할 빈캔을 제외한 총 빈캔 수를 알 수 있습니다:

73 = 알리사가 모은 빈캔 + 43

이제 알리사가 모은 빈캔의 수를 구합니다:

알리사가

## DPO 성능 평가

In [28]:
import torch
import torch.nn.functional as F

# prompt에 대해 response가 얼마나 그럴듯한지(평균 log_prob) 계산하는 함수
def calculate_log_prob(model, tokenizer, prompt, response):
    """
    주어진 prompt에 대한 response의 log probability(평균)를 계산합니다.

    Args:
        model: HuggingFace AutoModelForCausalLM (or similar)
        tokenizer: HuggingFace AutoTokenizer
        prompt (str): 프롬프트 텍스트 
        response (str): 응답 텍스트

    Returns:
        float: response 토큰들의 평균 log probability
    """
    full_text = prompt + " " + response  # prompt + 답변
    inputs = tokenizer(
        full_text,
        return_tensors='pt',
        truncation=True,      # 최대 글자 길이 이상시 자름
        max_length=512        # 최대 토큰 길이
    ).to(model.device)  # 토큰화시키고 GPU/CPU로 이동
    
    input_ids = inputs['input_ids']  # 토큰 id들
    attention_mask = inputs['attention_mask']  # 패딩 무시용 마스크

    # prompt만 따로 토큰화
    prompt_tokens = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    prompt_len = prompt_tokens["input_ids"].shape[1]  # prompt 토큰 길이 (어디서부터 response인지 경계)

    # 추론 모드(그래프 미생성 / 계산만 진행) : 메모리 절약
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # 순전파
        logits = outputs.logits  # 각 위치별로 다음 토큰 후보 점수표
    
    shift_logits = logits[..., :-1, :].contiguous()  # 마지막 위치 제외(다음 토큰 예측용 길이 맞추기 위해서)
    shift_labels = input_ids[..., 1:].contiguous()   # 정답 토큰은 한칸 왼쪽으로 당김 (=다음 토큰)

    log_probs = F.log_softmax(shift_logits, dim=-1)  # 점수표 -> log 확률포로 변환

    true_log_probs = torch.gather(log_probs, 2, shift_labels.unsqueeze(-1)).squeeze(-1)  # (batch, seq_len-1, 1) -> (batch, seq_len-1)

    seq_len = shift_labels.shape[1]  # 실제 점수 길이 (seq_len-1)
    if prompt_len >= seq_len:        # prompt가 너무 길어서 response 구간이 거의 없을 때
        valid_log_probs = true_log_probs[:, -1:]  # 마지막 차원값만 사용 (예외 처리)
    else:
        start_idx = max(0, prompt_len -1)  # response 첫 토큰 점수 위치(= prompt의 마지막 토큰 자리)
        valid_log_probs = true_log_probs[:, start_idx:]  # prompt 점수 제외하고 response 점수만 사용
    
    avg_log_probs = valid_log_probs.mean().item()  # response 점수 평균

    return avg_log_probs  # 평균 log_prob 반환

In [29]:
def calcualte_preference_accuracy(model, tokenizer, dataset, num_samples=100):
    """
    모델이 선호(chosen) 답변에 비선호(rejected) 답변보다 더 높은 확률을 부여하는지를 평가
    """
    correct = 0  # chosen을 더 높게 준 횟수
    total = min(num_samples, len(dataset))  # 평가 샘플 수

    print(f"{total}개 샘플에 대해 정확도 계산 시작")

    model.eval()  # 평가모드

    for idx in range(total):
        example = dataset[idx]  # 한개의 샘플씩 가져옴

        prompt = example['prompt']      # 프롬프트
        chosen = example['chosen']      # 선호답변
        rejected = example['rejected']  # 비선호답변

        chosen_score = calculate_log_prob(model, tokenizer, prompt, chosen)      # 선호답변 평균 log_prob
        rejected_score = calculate_log_prob(model, tokenizer, prompt, rejected)  # 비선호답변 평균 log_prob

        if chosen_score > rejected_score:  # 선호답변 평균이 비선호답변 평균보다 높으면
            correct += 1

        if (idx + 1) % 10 == 0:  # 10개 샘플마다
            print(f' - 진행률 : {idx+1}/{total} (현재 정확도: {correct/(idx+1) * 100:.2f}%)')
    
    accuracy = correct / total  # 최종 정확도
    return accuracy

test_accuracy = calcualte_preference_accuracy(model, tokenizer, test_dataset, num_samples=300)  # 테스트 셋으로 300개 평간
print(test_accuracy)

300개 샘플에 대해 정확도 계산 시작
 - 진행률 : 10/300 (현재 정확도: 70.00%)
 - 진행률 : 20/300 (현재 정확도: 70.00%)
 - 진행률 : 30/300 (현재 정확도: 80.00%)
 - 진행률 : 40/300 (현재 정확도: 75.00%)
 - 진행률 : 50/300 (현재 정확도: 76.00%)
 - 진행률 : 60/300 (현재 정확도: 71.67%)
 - 진행률 : 70/300 (현재 정확도: 70.00%)
 - 진행률 : 80/300 (현재 정확도: 71.25%)
 - 진행률 : 90/300 (현재 정확도: 73.33%)
 - 진행률 : 100/300 (현재 정확도: 73.00%)
 - 진행률 : 110/300 (현재 정확도: 72.73%)
 - 진행률 : 120/300 (현재 정확도: 74.17%)
 - 진행률 : 130/300 (현재 정확도: 74.62%)
 - 진행률 : 140/300 (현재 정확도: 75.71%)
 - 진행률 : 150/300 (현재 정확도: 75.33%)
 - 진행률 : 160/300 (현재 정확도: 76.25%)
 - 진행률 : 170/300 (현재 정확도: 77.06%)
 - 진행률 : 180/300 (현재 정확도: 76.11%)
 - 진행률 : 190/300 (현재 정확도: 76.32%)
 - 진행률 : 200/300 (현재 정확도: 77.50%)
 - 진행률 : 210/300 (현재 정확도: 77.62%)
 - 진행률 : 220/300 (현재 정확도: 77.73%)
 - 진행률 : 230/300 (현재 정확도: 77.39%)
 - 진행률 : 240/300 (현재 정확도: 78.33%)
 - 진행률 : 250/300 (현재 정확도: 77.60%)
 - 진행률 : 260/300 (현재 정확도: 78.46%)
 - 진행률 : 270/300 (현재 정확도: 78.89%)
 - 진행률 : 280/300 (현재 정확도: 77.86%)
 - 진행률 : 290/300 (현재 정확도: 77.93%)
 